# Comparing the TCN's for human activity classification with other Classifiers

# Note: We use a much simpler workflow: We don't do careful validation split, and don't repeat experiments to save the training time. This is just to illustrate a wider picture

# A. First let us train a simple CNN to do the job for comparison

- Dataset: https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones
- Video of process: https://www.youtube.com/watch?v=XOEN9W05_4A
- Observations at 50 Hz (i.e. 50 points per second)

### Released dataset details
- Pre-processing accelerometer and gyroscope using noise filters.
- Splitting data into fixed windows of 2.56 seconds (128 data points) with 50% overlap.
- Segregation of accelerometer data into gravity and motion components.
- The dataset was split into train (70%) and test (30%) sets based on data for subjects.

### 1D Convolutional Neural Network
- The model learns to extract features from intertial sensors and maps them to different activities.

In [96]:
from tensorflow import keras

In [97]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical

In [6]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot

### Data loading utilities

In [7]:
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values
 
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        loaded.append(data)
        
    loaded = dstack(loaded)
    return loaded

In [11]:
# load train or test
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
    # body acceleration
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    # body gyroscope
    filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y

In [16]:
# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix + 'datasets/UCI-HAR-Dataset/')
    
    # load all test
    testX, testy = load_dataset_group('test', prefix + 'datasets/UCI-HAR-Dataset/')
    
    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy

In [17]:
# Load the data
trainX, trainy, testX, testy = load_dataset()

(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)


### Let us play with the model

In [35]:
def get_model_cnn(n_timesteps, n_features):
    model = Sequential()
    model.add(Conv1D(filters=16, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(filters=16, kernel_size=3, activation='relu'))
    model.add(Dropout(0.3))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [36]:
verbose, epochs, batch_size = 1, 25, 32
n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]

model = get_model_cnn(n_timesteps, n_features)

In [37]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_7 (Conv1D)            (None, 126, 16)           448       
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 124, 16)           784       
_________________________________________________________________
dropout_4 (Dropout)          (None, 124, 16)           0         
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 62, 16)            0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 992)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 64)                63552     
_________________________________________________________________
dense_8 (Dense)              (None, 6)                

In [38]:
model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)

Epoch 1/25
7352/7352 [==============================] - 2s 210us/step - loss: 0.5706 - accuracy: 0.7741
Epoch 2/25
7352/7352 [==============================] - 1s 182us/step - loss: 0.2056 - accuracy: 0.9217
Epoch 3/25
7352/7352 [==============================] - 1s 183us/step - loss: 0.1456 - accuracy: 0.9421
Epoch 4/25
7352/7352 [==============================] - 1s 184us/step - loss: 0.1301 - accuracy: 0.9471
Epoch 5/25
7352/7352 [==============================] - 1s 179us/step - loss: 0.1137 - accuracy: 0.9535
Epoch 6/25
7352/7352 [==============================] - 1s 178us/step - loss: 0.1089 - accuracy: 0.9518
Epoch 7/25
7352/7352 [==============================] - 1s 177us/step - loss: 0.1054 - accuracy: 0.9546
Epoch 8/25
7352/7352 [==============================] - 1s 177us/step - loss: 0.1084 - accuracy: 0.9529
Epoch 9/25
7352/7352 [==============================] - 1s 180us/step - loss: 0.1092 - accuracy: 0.9523
Epoch 10/25
7352/7352 [==============================] - 1s 186u

In [39]:
# evaluate model
_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)

In [40]:
print('Test accuracy of model is:', accuracy)

Test accuracy of model is: 0.8802171945571899


# B. Let us train a  simple TCN 

In [74]:
import tensorflow as tf
from tensorflow.keras import Model

In [84]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, MaxPooling1D, Flatten
from tcn import TCN

In [85]:
def get_model_tcn(n_timesteps, n_features, nb_filters, n_outputs):
    model = Sequential()
    model.add(TCN(input_shape=(n_timesteps, n_features),
        nb_filters=nb_filters,
        kernel_size=3,
        nb_stacks=1,
        use_skip_connections=False,
        use_batch_norm=False,
        use_weight_norm=False,
        use_layer_norm=False))
    
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
   
    return model

In [102]:
verbose, epochs, batch_size = 1, 25, 32
n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]

model = get_model_tcn(n_timesteps, n_features, 16, n_outputs)

In [88]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
tcn_12 (TCN)                 (None, 16)                9232      
_________________________________________________________________
flatten (Flatten)            (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                1088      
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 390       
Total params: 10,710
Trainable params: 10,710
Non-trainable params: 0
_________________________________________________________________


In [89]:
model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)

Epoch 1/25
7352/7352 [==============================] - 6s 752us/sample - loss: 0.2066 - acc: 0.9195
Epoch 2/25
7352/7352 [==============================] - 5s 742us/sample - loss: 0.1529 - acc: 0.9353
Epoch 3/25
7352/7352 [==============================] - 6s 752us/sample - loss: 0.1301 - acc: 0.9479
Epoch 4/25
7352/7352 [==============================] - 6s 753us/sample - loss: 0.1314 - acc: 0.9460
Epoch 5/25
7352/7352 [==============================] - 5s 716us/sample - loss: 0.1213 - acc: 0.9470
Epoch 6/25
7352/7352 [==============================] - 5s 722us/sample - loss: 0.1126 - acc: 0.9520
Epoch 7/25
7352/7352 [==============================] - 6s 773us/sample - loss: 0.1160 - acc: 0.9472
Epoch 8/25
7352/7352 [==============================] - 6s 799us/sample - loss: 0.1130 - acc: 0.9502
Epoch 9/25
7352/7352 [==============================] - 5s 731us/sample - loss: 0.1193 - acc: 0.9490
Epoch 10/25
7352/7352 [==============================] - 5s 722us/sample - loss: 0.1020 - a

In [90]:
# evaluate model
_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)

print('Test accuracy of model is:', accuracy)

Test accuracy of model is: 0.88598573


# C. Let us train a very simple CNN 

In [107]:
def get_model_cnn_small(n_timesteps, n_features):
    model = Sequential()
    model.add(Conv1D(filters=12, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
    model.add(MaxPooling1D(pool_size=3))
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [108]:
verbose, epochs, batch_size = 1, 25, 32
n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]

model = get_model_cnn_small(n_timesteps, n_features)
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 126, 12)           336       
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 42, 12)            0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 504)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 32)                16160     
_________________________________________________________________
dense_11 (Dense)             (None, 6)                 198       
Total params: 16,694
Trainable params: 16,694
Non-trainable params: 0
_________________________________________________________________


In [109]:
model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)

Epoch 1/25
7352/7352 [==============================] - 1s 122us/sample - loss: 0.7146 - acc: 0.7088
Epoch 2/25
7352/7352 [==============================] - 1s 102us/sample - loss: 0.3067 - acc: 0.8915
Epoch 3/25
7352/7352 [==============================] - 1s 86us/sample - loss: 0.2097 - acc: 0.9264
Epoch 4/25
7352/7352 [==============================] - 1s 89us/sample - loss: 0.1707 - acc: 0.9361
Epoch 5/25
7352/7352 [==============================] - 1s 85us/sample - loss: 0.1480 - acc: 0.9429
Epoch 6/25
7352/7352 [==============================] - 1s 85us/sample - loss: 0.1359 - acc: 0.9446
Epoch 7/25
7352/7352 [==============================] - 1s 84us/sample - loss: 0.1229 - acc: 0.9514
Epoch 8/25
7352/7352 [==============================] - 1s 84us/sample - loss: 0.1166 - acc: 0.9528
Epoch 9/25
7352/7352 [==============================] - 1s 84us/sample - loss: 0.1133 - acc: 0.9536
Epoch 10/25
7352/7352 [==============================] - 1s 92us/sample - loss: 0.1123 - acc: 0.95

In [110]:
# evaluate model
_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)

print('Test accuracy of model is:', accuracy)

Test accuracy of model is: 0.88496774
